In [120]:
#from itertools import product
#from matplotlib.collections import LineCollection
#import statsmodels.formula.api as smf
#from scipy.stats import chisquare
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import Image, display, HTML
import pandas as pd
import datetime as datetime
movies_df = pd.read_csv('movies_for_analysis.csv',encoding='utf-8',index_col='imdb_id')
#movies.shape
#movies.sample(1)
#x.columns
movies_df['released'] = pd.to_datetime(movies_df['released'])
print('Dataset loaded')

Dataset loaded


In [121]:
display(HTML('<h1>Number of movies</h1>'))

print('Total number of movies: {0:,.0f}'.format(len(movies_df)))
print('Movies with Bechdel test results: {0:,.0f}'.format(len(movies_df[~pd.isnull(movies_df['bechdel_rating'])])))
print('Movies with box office receipt data: {0:,.0f}'.format(len(movies_df[~pd.isnull(movies_df['boxoffice_adj'])])))
print('Movies with profit data: {0:,.0f}'.format(len(movies_df[~pd.isnull(movies_df['profit_adj'])])))


Total number of movies: 7,266
Movies with Bechdel test results: 7,266
Movies with box office receipt data: 4,594
Movies with profit data: 2,748


In [123]:
bechdel_result_table = pd.DataFrame()
bechdel_result_table['nMovies'] = movies_df.groupby('bechdel_rating').size()
bechdel_result_table['result'] = ''
bechdel_result_table['result'] = np.where(bechdel_result_table.index == 0, 'Fewer than two women',bechdel_result_table['result'])
bechdel_result_table['result'] = np.where(bechdel_result_table.index == 1, 'Women don\'t talk',bechdel_result_table['result'])
bechdel_result_table['result'] = np.where(bechdel_result_table.index == 2, 'Women talk about a man',bechdel_result_table['result'])
bechdel_result_table['result'] = np.where(bechdel_result_table.index == 3, 'Passes Bechdel test',bechdel_result_table['result'])

bechdel_result_table['percentage'] = 100 * (bechdel_result_table['nMovies'] / len(movies_df))
#bechdel_result_table = bechdel_result_table.set_index('result')
#bechdel_result_table = bechdel_result_table.reset_index()
bechdel_result_table[['result','nMovies','percentage']].style.format({'result': '{:10}', 'nMovies': '{:,.0f}', 'percentage': '{0:.0f}%'})

,result,nMovies,percentage
bechdel_rating,,,
0,Fewer than two women,751,10%
1,Women don't talk,"1,607",22%
2,Women talk about a man,733,10%
3,Passes Bechdel test,"4,175",57%


In [142]:
#movies_df.sample(1).transpose()
movies_df['yearbin'] = ''
movies_df['yearbin'] = np.where((movies_df['bechdel_year'] < 1950), ' Before 1950', movies_df['bechdel_year'])
movies_df['yearbin'] = np.where(((movies_df['bechdel_year'] >= 1950) & (movies_df['bechdel_year'] < 1960)), '1950s', movies_df['yearbin'])
movies_df['yearbin'] = np.where(((movies_df['bechdel_year'] >= 1960) & (movies_df['bechdel_year'] < 1970)), '1960s', movies_df['yearbin'])
movies_df['yearbin'] = np.where(((movies_df['bechdel_year'] >= 1970) & (movies_df['bechdel_year'] < 1980)), '1970s', movies_df['yearbin'])
movies_df['yearbin'] = np.where(((movies_df['bechdel_year'] >= 1980) & (movies_df['bechdel_year'] < 1990)), '1980s', movies_df['yearbin'])
movies_df['yearbin'] = np.where(((movies_df['bechdel_year'] >= 1990) & (movies_df['bechdel_year'] < 2000)), '1990s', movies_df['yearbin'])
movies_df['yearbin'] = np.where(((movies_df['bechdel_year'] >= 2000) & (movies_df['bechdel_year'] < 2010)), '2000s', movies_df['yearbin'])
movies_df['yearbin'] = np.where(((movies_df['bechdel_year'] >= 2010)), '2010s', movies_df['yearbin'])

movies_df.groupby('yearbin').size()

yearbin
 Before 1950     541
1950s            240
1960s            303
1970s            324
1980s            545
1990s            958
2000s           2033
2010s           2322
dtype: int64